In [1]:
import os 
from dotenv import load_dotenv, find_dotenv
import psycopg2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

2/8/17

In this notebook I test census data tables imported into my ec2 development db for the Hack OR Emergency Response team project.

In [2]:
# walk root diretory to find and load .env file w/ AWS host, username and password
load_dotenv(find_dotenv())

True

In [4]:
# connect to postgres
def pgconnect():
    try:
        conn = psycopg2.connect(database=os.environ.get("database"), user=os.environ.get("user"), 
                            password = os.environ.get("password"), 
                            host=os.environ.get("host"), port=os.environ.get("port"))
        print("Opened database successfully")
        return conn
    
    except psycopg2.Error as e:
        print("I am unable to connect to the database")
        print(e)
        print(e.pgcode)
        print(e.pgerror)
        print(traceback.format_exc())
        return None

In [5]:
def pquery(QUERY):
    '''
    takes SQL query string, opens a cursor, and executes query in psql
    '''
    conn = pgconnect()
    cur = conn.cursor()
    
    try:
        print("SQL QUERY = "+QUERY)
        cur.execute("SET statement_timeout = 0")
        cur.execute(QUERY)
        # Extract the column names and insert them in header
        col_names = []
        for elt in cur.description:
            col_names.append(elt[0])    
    
        D = cur.fetchall() #convert query result to list
        # Create the dataframe, passing in the list of col_names extracted from the description
        return pd.DataFrame(D, columns=col_names)

    except Exception as e:
        print(e.pgerror)
            
    finally:
        conn.close()


In [6]:
# get all rows from census block to fma lookup table 
QUERY1='''SELECT *
FROM fmac_proportion;
'''

In [7]:
df1 = pquery(QUERY1)

Opened database successfully
SQL QUERY = SELECT *
FROM fmac_proportion;



In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770 entries, 0 to 769
Data columns (total 4 columns):
c_block        770 non-null object
fma            770 non-null object
overlap_cbg    770 non-null float64
overlap_fma    770 non-null float64
dtypes: float64(2), object(2)
memory usage: 24.1+ KB


In [9]:
df1.head(25)

,c_block,fma,overlap_cbg,overlap_fma
0,410050201001,10,0.002468,7.874073e-04
1,410050203032,10,0.000015,7.926580e-07
2,410050203041,18,0.000056,4.442453e-06
3,410050208001,10,0.001001,2.550445e-04
4,410050208001,20,0.055454,1.285901e-02
5,410050209001,20,0.239613,8.783360e-03
6,410050210001,25,0.003061,1.808271e-04
7,410050210002,20,0.006363,6.558618e-04
8,410050210002,25,0.015021,1.445884e-03
9,410050212001,20,0.042128,4.428812e-03


Each row in this lookup table is an intersection of census block group (c_block) to Fire Management Area (fma). 

* overlap_cbg is the proprtion calculated by intersection area / total census block area

* overlap_fma is the proportion calculated by intersection area / total fma area



**Now I test some queries converting data by census block group to fma, weighting by the sum of intersecion proportions (overlap_cbg)**

In [11]:
# try joining census total population by census block group to fma
QUERY2='''SELECT f.fma,
  round(sum(c.estimate_total*f.overlap_cbg)) AS fma_population_total
FROM fmac_proportion f INNER JOIN total_population c
ON f.c_block = c.id2
GROUP BY f.fma
ORDER BY fma_population_total DESC
'''

In [12]:
df2 = pquery(QUERY2)

Opened database successfully
SQL QUERY = SELECT f.fma,
  round(sum(c.estimate_total*f.overlap_cbg)) AS fma_population_total
FROM fmac_proportion f INNER JOIN total_population c
ON f.c_block = c.id2
GROUP BY f.fma
ORDER BY fma_population_total DESC



In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 2 columns):
fma                     31 non-null object
fma_population_total    31 non-null float64
dtypes: float64(1), object(1)
memory usage: 576.0+ bytes


In [14]:
df2

,fma,fma_population_total
0,07,42544.0
1,25,41133.0
2,14,37634.0
3,09,34321.0
4,11,34236.0
5,31,31487.0
6,19,30588.0
7,28,28355.0
8,18,27347.0
9,29,24628.0


In [18]:
# try joining census housing_tenure by census block group to fma
QUERY3='''SELECT f.fma,
  round(sum(c.estimate_total*f.overlap_cbg)) AS total_households,
  round(sum(c.estimate_total_owner_occupied*f.overlap_cbg)) AS total_owner_occupied,
  round(sum(c.estimate_total_renter_occupied*f.overlap_cbg)) AS total_renter_occupied
FROM fmac_proportion f INNER JOIN housing_tenure c
ON f.c_block = c.id2
GROUP BY f.fma
ORDER BY total_households DESC
'''

In [19]:
df3 = pquery(QUERY3)

Opened database successfully
SQL QUERY = SELECT f.fma,
  round(sum(c.estimate_total*f.overlap_cbg)) AS total_households,
  round(sum(c.estimate_total_owner_occupied*f.overlap_cbg)) AS total_owner_occupied,
  round(sum(c.estimate_total_renter_occupied*f.overlap_cbg)) AS total_renter_occupied
FROM fmac_proportion f INNER JOIN housing_tenure c
ON f.c_block = c.id2
GROUP BY f.fma
ORDER BY total_households DESC



In [20]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 4 columns):
fma                      31 non-null object
total_households         31 non-null float64
total_owner_occupied     31 non-null float64
total_renter_occupied    31 non-null float64
dtypes: float64(3), object(1)
memory usage: 1.0+ KB


In [21]:
df3

,fma,total_households,total_owner_occupied,total_renter_occupied
0,25,17263.0,9648.0,7615.0
1,09,15456.0,8071.0,7385.0
2,07,14879.0,6141.0,8738.0
3,14,14326.0,9551.0,4775.0
4,03,14052.0,3640.0,10412.0
5,19,12574.0,6989.0,5585.0
6,11,12245.0,6399.0,5847.0
7,28,11831.0,7678.0,4153.0
8,18,11570.0,7581.0,3989.0
9,13,11457.0,4717.0,6741.0


In [22]:
# try joining census estimate_median_household_income by census block group to fma
# note this is not a statistically valid query since we can't simply apply a weighted average to medians
# but it's at least one approach to approximate a median for fmas

QUERY4='''SELECT f.fma,
  round(sum(h.estimate_median_household_income*f.overlap_fma)) AS median_household_income,
  round(sum(h.moe*f.overlap_fma)) AS moe
FROM fmac_proportion f INNER JOIN household_income_median h
ON f.c_block = h.id2
GROUP BY f.fma
ORDER BY median_household_income DESC
'''

In [23]:
df4 = pquery(QUERY4)

Opened database successfully
SQL QUERY = SELECT f.fma,
  round(sum(h.estimate_median_household_income*f.overlap_fma)) AS median_household_income,
  round(sum(h.moe*f.overlap_fma)) AS moe
FROM fmac_proportion f INNER JOIN household_income_median h
ON f.c_block = h.id2
GROUP BY f.fma
ORDER BY median_household_income DESC



In [24]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 3 columns):
fma                        31 non-null object
median_household_income    31 non-null float64
moe                        31 non-null float64
dtypes: float64(2), object(1)
memory usage: 824.0+ bytes


In [25]:
df4

,fma,median_household_income,moe
0,27,137883.0,46428.0
1,16,130537.0,42369.0
2,15,117597.0,30528.0
3,10,108234.0,28216.0
4,05,90861.0,19691.0
5,03,90696.0,26759.0
6,06,84673.0,15473.0
7,18,81527.0,18914.0
8,22,72752.0,14619.0
9,09,71971.0,20202.0
